# ロジスティック回帰を実装する

In [1]:
import tensorflow as tf

## Inferance
与えられたミニバッチに対してクラスの出力を返す

In [2]:
def inference(x):
    # 
    init = tf.constant_initializer(value=0)
    W = tf.get_variable("W", [784,10], initializer = init)
    b  = tf.get_variable("b", [10], initializer = init)
    output = tf.nn.softmax(tf.matmul(x, W) + b)
    
    # 表示用
    tf.summary.histogram("weights",W)
    tf.summary.histogram("biases",b)
    tf.summary.histogram("output",output)

    return output

## Loss
損失関数（クロスエントロピー）

$loss = - \sum y_k log{o_k}$

reduce_sum : 与えられたリストの総和をとる

reduce_mean : 与えられたリストの平均を返す

In [3]:
def loss(output, y):
    elementwise_product = y * tf.log(output)
    
    xentropy = -tf.reduce_sum(elementswise_product, reduction_indices=1)
    
    loss = tf.reduce_mean(xentropy)
    
    return loss

## Training
パラメータの勾配を計算し，更新を行う

In [4]:
def training(cost, global_step):
    tf.summary.scalar("cost",cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    return tain_op

## Evaluate


In [5]:
def evaluate(output, y):
    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
    accuracy = tf.resuce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.acalar("valisation error", (1.0 -accuracy))
    return accuracy

## Main
MNISTの手書き数字データを利用して分類を行う

In [ ]:
if __name__ == "__main__":
    # logが存在しているなら削除
    if os.path.exists("logistic_logs"):
        shutil.rmtree("logistic_logs")
    
    with tf.Graph().as_default():
        x = tf.placeholder("float", [None,784]) # image = 28*28  = 784
        y = tf.placeholder("float", [None,10]) # 0~9 classes
        output = inference(x)
        cost = loss(output, y)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        train_op = training(cost, global_step)
        eval_op = evaluate(output, y)
        # merge graphs
        summary_op =  tf.summary.merge_all()
        # model save
        saver = tf.train.Saver()
        sess = tf.Session()
        # log write
        summary_writer = tf.summary.FileWriter(
            "logistic_logs",
            graph_def=sess.graph_def
        )
        intit_op = tf.global_variables_initalizer()
        sess.run(init_op)
        
        # training cycle